In [4]:
import base

import pandas as pd
import json
from collections import namedtuple
import re
import ngender
import time


In [2]:
# my private modules
import fund_scanner.database
import fund_scanner.readurl

In [6]:
#page=page_index,items_in_one_page

url = 'http://fund.eastmoney.com/Data/FundRankScale.aspx?v=%f'%time.time()

body = fund_scanner.readurl.read_from_url(url)


In [21]:
insider = re.search('\[\[(.*?)\]\]', body).group(1)
insider = insider.replace('\'', '"')
insider = '{"main":[['+insider+']]}'
obj = json.loads(insider, object_hook=lambda d: namedtuple('X', d.keys())(*d.values()))
df =pd.DataFrame(obj.main)
df.sort_values(by=2)
df = df.loc[:,[0,1,2,7,9]]
df.columns = ['companies_code', 'companies_name_full', 'companies_start_date', 'companies_size', 'companies_name_short']

df

,companies_code,companies_name_full,companies_start_date,companies_size,companies_name_short
0,80036782,招商基金管理有限公司,2002-12-27,3713.23,招商基金
1,80351991,鑫元基金管理有限公司,2013-08-29,367.37,鑫元基金
2,80175498,英大基金管理有限公司,2012-08-17,21.85,英大基金
3,80068180,诺德基金管理有限公司,2006-06-08,70.47,诺德基金
4,80050229,上投摩根基金管理有限公司,2004-05-12,1229.57,上投摩根基金
5,80037023,华富基金管理有限公司,2004-04-19,420.21,华富基金
6,80366080,上银基金管理有限公司,2013-08-30,372.44,上银基金
7,80404004,长江证券(上海)资产管理有限公司,2014-09-16,54.21,长江证券(上海)
8,80380794,创金合信基金管理有限公司,2014-07-09,155.66,创金合信基金
9,80488954,富荣基金管理有限公司,2016-01-25,25.49,富荣基金


In [26]:
with fund_scanner.database.get_connection() as cursor:

    for index, row in df.iterrows():
        #print(row['净值日期'])
        sql = 'INSERT INTO companies '+\
        '(companies_name_full, companies_name_short, companies_code, companies_start_date, companies_size) '+\
        'VALUES (%s,%s,%s,%s,%s) ON DUPLICATE KEY UPDATE '+\
        'companies_name_full=%s, companies_name_short=%s, companies_size=%s;'

        cursor.execute(
            sql, (
                row['companies_name_full'],
                row['companies_name_short'], 
                row['companies_code'],
                row['companies_start_date'], 
                base.extract_float(row, 'companies_size'),
                row['companies_name_full'], 
                row['companies_name_short'],
                base.extract_float(row, 'companies_size')
            )
        )



